In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch

In [3]:
import numpy as np

In [4]:
import torchvision as tv

In [5]:
import pandas as pd
import numpy as np

In [6]:
import time

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
print(device)

cuda


In [9]:
BATCH_SIZE=256

In [10]:
train_dataset = tv.datasets.FashionMNIST('.', train=True, transform=tv.transforms.ToTensor(), download=True)
test_dataset = tv.datasets.FashionMNIST('.', train=False, transform=tv.transforms.ToTensor(), download=True)
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [11]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

## Модель

In [12]:
model = torch.nn.Sequential(
    
    torch.nn.Flatten(),    
    
    torch.nn.Linear(784, 512),  
    torch.nn.BatchNorm1d(512),
    torch.nn.ReLU(),             
    
    torch.nn.Linear(512, 256), 
    torch.nn.Dropout(p=0.2),  
    torch.nn.ReLU(),       
    
    torch.nn.Linear(256, 128),  
    
    torch.nn.ReLU(),     
    torch.nn.BatchNorm1d(128), 
    
    torch.nn.Linear(128, 10)
)

model = model.to(device)
print(f"Model is on device: {next(model.parameters()).device}")

Model is on device: cuda:0


In [13]:
loss = torch.nn.CrossEntropyLoss()
num_epochs = 10

In [14]:
def train_model():
    for ep in range(num_epochs):
        train_iters, train_passed  = 0, 0
        train_loss, train_acc = 0., 0.
        start=time.time()
        
        model.train()
        for X, y in train:
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_pred = model(X)            
            l = loss(y_pred, y)
            l.backward()
            trainer.step()
            train_loss += l.item()
            train_acc += (y_pred.argmax(dim=1) == y).sum().item()
            train_iters += 1
            train_passed += len(X)
        
        test_iters, test_passed  = 0, 0
        test_loss, test_acc = 0., 0.
        model.eval()
        for X, y in test:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            l = loss(y_pred, y)
            test_loss += l.item()
            test_acc += (y_pred.argmax(dim=1) == y).sum().item()
            test_iters += 1
            test_passed += len(X)
            
        print("ep: {}, taked: {:.4}, train_loss: {:.4f}, train_acc: {:.4f}, test_loss: {:.4f}, test_acc: {:.4f}".format(
            ep, time.time() - start, train_loss / train_iters, train_acc / train_passed,
            test_loss / test_iters, test_acc / test_passed)
        )

In [15]:
trainer = torch.optim.Adam(model.parameters(), lr=.0001)
train_model()

ep: 0, taked: 4.917, train_loss: 0.6714, train_acc: 0.7927, test_loss: 0.4526, test_acc: 0.8416
ep: 1, taked: 3.829, train_loss: 0.3952, train_acc: 0.8683, test_loss: 0.3887, test_acc: 0.8593
ep: 2, taked: 3.866, train_loss: 0.3358, train_acc: 0.8837, test_loss: 0.3629, test_acc: 0.8689
ep: 3, taked: 3.922, train_loss: 0.2991, train_acc: 0.8953, test_loss: 0.3531, test_acc: 0.8700
ep: 4, taked: 3.812, train_loss: 0.2693, train_acc: 0.9063, test_loss: 0.3439, test_acc: 0.8759
ep: 5, taked: 3.815, train_loss: 0.2458, train_acc: 0.9124, test_loss: 0.3392, test_acc: 0.8755
ep: 6, taked: 3.816, train_loss: 0.2222, train_acc: 0.9223, test_loss: 0.3293, test_acc: 0.8810
ep: 7, taked: 3.833, train_loss: 0.2046, train_acc: 0.9277, test_loss: 0.3238, test_acc: 0.8836
ep: 8, taked: 3.82, train_loss: 0.1855, train_acc: 0.9354, test_loss: 0.3269, test_acc: 0.8832
ep: 9, taked: 3.819, train_loss: 0.1686, train_acc: 0.9421, test_loss: 0.3350, test_acc: 0.8838
